In [1]:
import pandas as pd
import sys
import constants
from sklearn.model_selection import train_test_split
import numpy as np
import os

Read file

In [2]:
from preprocessing import transform_and_import_data
listings = transform_and_import_data(constants.DATA_PATH)

In [3]:
listings.set_index("id", inplace=True)
listings.drop(['name', 'description', "neighborhood_overview", "longitude", "latitude"], axis=1, inplace=True)

In [4]:
listings.head()

,host_response_time,host_is_superhost,host_listings_count,host_total_listings_count,host_identity_verified,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms_text,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable
id,,,,,,,,,,,,,,,,,,,,,
10595,within an hour,t,6.0,6.0,t,ΑΜΠΕΛΟΚΗΠΟΙ,Entire rental unit,Entire home/apt,8,2 baths,...,33,4.78,4.82,4.76,4.85,4.85,4.52,4.67,t,t
10990,within an hour,t,6.0,6.0,t,ΑΜΠΕΛΟΚΗΠΟΙ,Entire rental unit,Entire home/apt,4,1 bath,...,62,4.80,4.84,4.84,4.89,4.89,4.77,4.74,t,t
10993,within an hour,t,6.0,6.0,t,ΑΜΠΕΛΟΚΗΠΟΙ,Entire rental unit,Entire home/apt,2,1 bath,...,87,4.83,4.91,4.93,4.98,4.98,4.84,4.78,t,t
2219385,within an hour,f,2.0,2.0,t,ΣΤΑΔΙΟ,Entire condo,Entire home/apt,4,2 baths,...,29,4.62,4.57,4.41,4.75,4.79,4.61,4.64,t,t
10995,within an hour,t,6.0,6.0,t,ΑΜΠΕΛΟΚΗΠΟΙ,Entire rental unit,Entire home/apt,4,1 bath,...,29,4.83,4.96,4.93,4.93,4.89,4.81,4.81,t,t


Transform categorical, etc.

In [4]:
Y = listings["price"]
X = listings.drop(["price"], axis=1)

In [5]:
from pandas.api.types import is_numeric_dtype

numeric_cols = []
categorical_cols = []

for column in X.columns:
    if is_numeric_dtype(X[column]):
        numeric_cols.append(column)
    else:
        categorical_cols.append(column)

Impute and Encode categorical columns

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [8]:
X[categorical_cols] = SimpleImputer(strategy="most_frequent").fit_transform(X[categorical_cols])

In [9]:
for column in X[categorical_cols].columns:
    X[column] = LabelEncoder().fit_transform(X[column])

!pip install -U scikit-learn

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

cat_pipeline = Pipeline([
        ('cat_scaler', MinMaxScaler())
    ])

In [12]:
from sklearn.compose import ColumnTransformer

pipeline = ColumnTransformer([
        ("num", num_pipeline, numeric_cols),
        ("cat", cat_pipeline, categorical_cols)
    ])

In [13]:
#Split X, Y and train/test
# split the data into train and test set
# Split the data into training and test set
X_train_o, X_test, y_train_o, y_test = train_test_split(X, Y, test_size=0.2, random_state=24)

# Split the data into training and Validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_o, y_train_o, test_size=0.15, random_state=24)

print(len(X_train))
print(len(X_test))

7697
2265


In [14]:
X_train = pipeline.fit_transform(X_train)

X_val = pipeline.transform(X_val)
X_test = pipeline.transform(X_test)

In [21]:
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

In [ ]:
!pip install keras

In [24]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

In [38]:
framed_y_train = y_train.to_frame()

In [49]:
y_train.values

array([128,  75,  75, ..., 220,  99, 276], dtype=int64)

In [50]:
input_size=2
output_size=1
models = Sequential([
                        Dense(output_size)
                    ])
custom_optimizer= SGD(learning_rate=0.02)
models.compile(optimizer=custom_optimizer,loss='mean_squared_error')
models.fit(X, y_train.values, epochs=100,verbose=1)

ValueError: object __array__ method not producing an array

In [43]:
# Determine the input size of the algorithm
input_shape = 28

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=input_shape))
model.add(Dropout(0,2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# softmax classification
model.add(Dense(1, activation='linear'))
model.summary()

# Compile the model
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mse'])

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 64)                1856      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 128)               8320      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                

In [ ]:
# Callbacks
# es = EarlyStopping(monitor='val_accuracy', patience=8, min_delta=1)
# filepath="Skin_lesion_characterization/weights.best.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint] #PlotLossesKeras()

In [26]:
np.array(y_train)

array([128,  75,  75, ..., 220,  99, 276], dtype=int64)

In [25]:
batch_size = 64
epochs = 10

history_custom = model.fit(
    X_train, np.array(y_train),
    epochs=epochs,
    batch_size = batch_size,
    validation_data=(X_val, np.array(y_val)),
    verbose=1
    # callbacks=[PlotLossesKeras()]
    )

# save model

score = model.evaluate(pd.DataFrame(X_test), y_test)
print(score)
print('Test accuracy:', score[1])

ValueError: object __array__ method not producing an array

Evalution

model.evaluate



Adjust Model

In [ ]:
plt.subplot(1,2,1)
plt.plot(image_mobilenet.history['accuracy'], marker='o')
plt.plot(image_mobilenet.history['val_accuracy'], marker='o')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')

plt.subplot(1,2,2)
plt.plot(image_mobilenet.history['loss'], marker='o')
plt.plot(image_mobilenet.history['val_loss'], marker='o')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()